In [ ]:
import pandas as pd
import json

: 

In [2]:
tmdb_df = pd.read_csv("TMDB_movie_dataset_v11.csv")
tmdb_df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [3]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324665 entries, 0 to 1324664
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1324665 non-null  int64  
 1   title                 1324648 non-null  object 
 2   vote_average          1324665 non-null  float64
 3   vote_count            1324665 non-null  int64  
 4   status                1324665 non-null  object 
 5   release_date          1055145 non-null  object 
 6   revenue               1324665 non-null  int64  
 7   runtime               1324665 non-null  int64  
 8   adult                 1324665 non-null  bool   
 9   backdrop_path         336546 non-null   object 
 10  budget                1324665 non-null  int64  
 11  homepage              137477 non-null   object 
 12  imdb_id               646809 non-null   object 
 13  original_language     1324665 non-null  object 
 14  original_title        1324648 non-

In [4]:
# Keep only released + non-adult movies
tmdb_df = tmdb_df[(tmdb_df["status"] == "Released") & (tmdb_df["adult"] == False)]

# Select relevant columns
columns_to_keep = [
    "id", "title", "overview", "genres", "original_language", "tagline",
    "keywords", "runtime", "release_date", "popularity", "poster_path"
]

tmdb_df = tmdb_df[columns_to_keep]

In [5]:
# Load language mapping json
with open("languages.json", "r") as f:
    language_map = json.load(f)

major_movie_languages = [
    "en",
    "hi", "te", "ta", "ml", "kn", "bn", "mr", "or",
    "ko", "ja", "zh", "cn", "th", "id", "vi",
    "fr", "de", "it", "es", "nl",
    "sv", "da", "fi",
    "pl", "pt",
    "cs", "sk", "sl", "hr",
    "et", "lv"
]

# Filter rows to keep only the major languages and runtime >= 60 mins
tmdb_df = tmdb_df[tmdb_df["original_language"].isin(major_movie_languages)]
tmdb_df = tmdb_df[tmdb_df["runtime"] >= 60]

# Replace language code with full name
tmdb_df["original_language"] = tmdb_df["original_language"].map(language_map)

# Drop rows with missing title
tmdb_df["title"] = tmdb_df["title"].fillna("").astype(str).str.strip()
tmdb_df = tmdb_df[tmdb_df["title"] != ""]

# Clean overview (fill empty but don't drop)
tmdb_df["overview"] = tmdb_df["overview"].fillna("")

# Clean genres, keywords, tagline
for col in ["genres", "keywords", "tagline"]:
    tmdb_df[col] = tmdb_df[col].fillna("")

# Reset index
tmdb_df = tmdb_df.reset_index(drop=True)


In [6]:
# Convert release_date to NaT where invalid
tmdb_df["release_date"] = pd.to_datetime(tmdb_df["release_date"], errors="coerce")


In [7]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354344 entries, 0 to 354343
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 354344 non-null  int64         
 1   title              354344 non-null  object        
 2   overview           354344 non-null  object        
 3   genres             354344 non-null  object        
 4   original_language  354344 non-null  object        
 5   tagline            354344 non-null  object        
 6   keywords           354344 non-null  object        
 7   runtime            354344 non-null  int64         
 8   release_date       327722 non-null  datetime64[ns]
 9   popularity         354344 non-null  float64       
 10  poster_path        293041 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 29.7+ MB


In [8]:
missing_release_ids = tmdb_df[tmdb_df["release_date"].isna()]["id"].astype(int).tolist()

print("Missing release dates:", len(missing_release_ids))

with open("missing_release_ids.json", "w") as f:
    json.dump(missing_release_ids, f, indent=2)

print("Saved → missing_release_ids.json")

Missing release dates: 26622
Saved → missing_release_ids.json


In [9]:
# Load JSON
with open("missing_release_dates.json", "r") as f:
    release_map = json.load(f)

# Convert keys to ints
release_map = {int(k): v for k, v in release_map.items()}

# Create a pandas Series from the dictionary
release_series = pd.Series(release_map, name="release_date")

# Align by ID and update
tmdb_df = tmdb_df.set_index("id")
tmdb_df["release_date"].update(release_series)
tmdb_df = tmdb_df.reset_index()

# Convert to datetime
tmdb_df["release_date"] = pd.to_datetime(tmdb_df["release_date"], errors="coerce")

print("Remaining:", tmdb_df["release_date"].isna().sum())

/var/folders/0y/1f0qyvx12h59npx2jsm5_x940000gn/T/ipykernel_9489/1296922318.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tmdb_df["release_date"].update(release_series)


Remaining: 14675


In [ ]:
tmdb_df = tmdb_df.dropna(subset=["release_date"])
tmdb_df["release_year"] = tmdb_df["release_date"].dt.year
tmdb_df = tmdb_df[(tmdb_df["release_year"] >= 1950) & (tmdb_df["release_year"] <= 2025)].reset_index(drop=True)
tmdb_df = tmdb_df.drop(columns=["release_date"])

In [13]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318017 entries, 0 to 318016
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 318017 non-null  int64  
 1   title              318017 non-null  object 
 2   overview           318017 non-null  object 
 3   genres             318017 non-null  object 
 4   original_language  318017 non-null  object 
 5   tagline            318017 non-null  object 
 6   keywords           318017 non-null  object 
 7   runtime            318017 non-null  int64  
 8   popularity         318017 non-null  float64
 9   poster_path        268794 non-null  object 
 10  release_year       318017 non-null  int32  
dtypes: float64(1), int32(1), int64(2), object(7)
memory usage: 25.5+ MB


In [14]:
tmdb_df.to_parquet("tmdb_dataset_cleaned.parquet")